In [2]:
import re
from dataclasses import dataclass

import typing


In [61]:
@dataclass
class Almanach:
    ingredients : typing.Dict
    mappings : typing.Dict

    @classmethod
    def fromfilename(cls,filename):
        with open(filename) as f:
            chunks = f.read().split("\n\n")
            seeds  = cls._getseeds(chunks.pop(0))
            
            mappings = {}
            for chunk in chunks:
                _map = AlmanachMap.from_almanach_chunk(chunk)
                mappings[_map.source] =  _map
            return cls({"seed":seeds},mappings)

    @classmethod
    def _getseeds(cls,line):
        return [int(id) for id in line.split()[1:]]

    def fill_ingredients(self):
        source = "seed"
        while source != "location":
            mapping = self.mappings[source]
            sourceid = self.ingredients[source]
            destid = [mapping.map_single_id(id) for id in sourceid]
            self.ingredients[source:=mapping.destination] = destid


@dataclass
class RangeAlmanach(Almanach):

    @classmethod
    def _getseeds(cls,line):
        seeds = iter(line.split()[1:])
        return [(int(id),int(ran)) for id,ran in zip(seeds,seeds)]
    
    def fill_ingredients(self):
        source = "seed"
        while source != "location":
            mapping = self.mappings[source]
            sources= self.ingredients[source]

            destid =mapping.map_ranges(sources)

            self.ingredients[source:=mapping.destination] = destid


@dataclass
class AlmanachMap:
    source : str
    destination : str
    ranges : typing.Iterable

    def from_almanach_chunk(chunk):
        chunksplitted = chunk.splitlines()
        metadata = re.match("(.+)-to-(.+) map:",chunksplitted.pop(0))
        ranges = [[int(n) for n in line.split()] for line in chunksplitted]
        return AlmanachMap(metadata.group(1),metadata.group(2),ranges)
    
    def map_single_id(self,sourceid):
        for (deststart, sourcestart, rangelength) in self.ranges:
            delta = sourceid-sourcestart
            if delta>=0 and delta<rangelength:
                return deststart+delta
        return sourceid
    
    def map_ranges(self,sourceranges):
        tomap = sourceranges.copy()
        newranges = []
        while tomap:
            mapped = False
            (sourceid,sourcerange) = tomap.pop()
            for (deststart, sourcestart, delta) in self.ranges:
                low,high = max(sourceid,sourcestart),min(sourceid+sourcerange,sourcestart+delta)
                newdelta = high-low
                if newdelta>0:
                    mapped=True
                    offset = low-sourcestart
                    newranges.append((deststart+offset,newdelta))
                    if sourceid<sourcestart:
                        tomap.append((sourceid,sourcestart-sourceid))
                    if high<sourceid+sourcerange:
                        tomap.append((high,sourceid+sourcerange-high))
                    break
            if not mapped:
                newranges.append((sourceid,sourcerange))
        return newranges
    





    

# Part 1

In [57]:
almanach = Almanach.fromfilename("input5.txt")
almanach.fill_ingredients()

print(f"Smallest location: {min(almanach.ingredients['location'])}")

Smallest location: 486613012


# Part 2

In [63]:
almanach = RangeAlmanach.fromfilename("input5.txt")
almanach.fill_ingredients()

print(f"Smallest location: {min(almanach.ingredients['location'])[0]}")

Smallest location: 56931769
